In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import re
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import interp1d

def extract_time(file_path):
    with open(file_path, 'r') as file:
        first_colomn = []
        for row, line in enumerate(file):
            if row < 6:
                continue
            line = line.strip()
            colomns = line.split()
            if len(colomns) >= 5:
                first_colomn.append(float(colomns[0]))
    return first_colomn

def extractField_0(file_path):
    with open(file_path, 'r') as file:
        fifth_colomn = []
        for row, line in enumerate(file):
            if row < 6:
                continue
            line = line.strip()
            colomns = line.split()
            if len(colomns) >= 5:
                fifth_colomn.append(float(colomns[5]))
    return fifth_colomn

## TIPTOE short

In [46]:
df_final=pd.read_csv("/home/user/BachelorThesis/trecxcoefftests/tiptoe_short/0367/expec", sep='\s+', header=8)
#shift every column name by one and remove the first column
df_final.columns = df_final.columns[1:].tolist() + [""]
#remove last column
df_final = df_final.iloc[:, :-1]



df_normal = pd.read_csv("/home/user/BachelorThesis/trecxcoefftests/tiptoe_short/normal_trecx/expec", sep='\s+', header=8)
df_normal.columns = df_normal.columns[1:].tolist() + [""]
#remove last column
df_normal = df_normal.iloc[:, :-1]






fig = go.Figure()
fig.add_trace(go.Scatter(x=np.array(df_final["Time"]), y=np.array(df_final["<Occ{H0:2}>"]), mode='lines', name='c^2_Occ_new'))
fig.add_trace(go.Scatter(x=np.array(df_final["Time"]), y=np.array(df_final["Re{<H0:1|psi>}"]**2+df_final["Imag{<H0:1|psi>}"]**2), mode='lines', name='c^2_new'))


#fig.add_trace(go.Scatter(x=np.array(df_final["Time"]), y=np.array(df_final["Re{<H0:1|psi>}"]), mode='lines', name='c_Re_new'))
#fig.add_trace(go.Scatter(x=np.array(df_final["Time"]), y=np.array(df_final["<Occ{H0:2}>"]), mode='lines', name='c^2_Occ_new'))
#fig.add_trace(go.Scatter(x=np.array(df_final["Time"]), y=np.array(df_normal["<Occ{H0:1}>"]), mode='lines', name='c^2_old'))
#fig.add_trace(go.Scatter(x=np.array(df_final["Time"]), y=np.array(df_final["Re{<H0:1|psi>}"]**2+df_final["Imag{<H0:1|psi>}"]**2)-np.array(df_final["<Occ{H0:1}>"]), mode='lines', name='diff'))
#fig.add_trace(go.Scatter(x=np.array(df_final["Time"]), y=np.array(df_final["Imag{<H0:1|psi>}"]), mode='lines', name='c_Imag_new'))
# warum ist Re(H0:0|psi) gleich c^2_old? ich dachte in occ wird c^2 berechnet aberscheinabr nur Re(c)
fig.show()

## Weakpulse

In [47]:
df_weak=pd.read_csv("/home/user/BachelorThesis/trecxcoefftests/weakpulse/0043/expec", sep="\s+", header=8)
#shift every column name by one and remove the first column
df_weak.columns = df_weak.columns[1:].tolist() + [""]
#remove last column
df_weak = df_weak.iloc[:, :-1]



time0 = extract_time("/home/user/BachelorThesis/trecxcoefftests/weakpulse/0043/Laser")
field0 = extractField_0("/home/user/BachelorThesis/trecxcoefftests/weakpulse/0043/Laser")

integrand = np.exp(1j* 1/4 * np.array(time0)) * np.array(field0)*3
cumulative_integral = cumulative_trapezoid(-1j*integrand, x=np.array(time0), initial=0)
interp_func = interp1d(time0, cumulative_integral, kind='cubic', fill_value="extrapolate")
cumulative_integral_reduced = interp_func(df_weak["Time"])



fig = go.Figure()

fig.add_trace(go.Scatter(x=np.array(df_weak["Time"]), y=df_weak["<Occ{H0:1}>"], mode='lines', name='c^2_Occ_new'))
#fig.add_trace(go.Scatter(x=np.array(df_weak["Time"]), y=df_weak["Re{<H0:1|psi>}"]**2+df_weak["Imag{<H0:1|psi>}"]**2, mode='lines', name='c^2_new'))

fig.add_trace(go.Scatter(x=np.array(df_weak["Time"]), y=np.abs(cumulative_integral_reduced)**2, mode='lines', name='c^2_analytical'))

fig.show()

# TIPTOE dense

In [98]:
df_dense=pd.read_csv("/home/user/BachelorThesis/trecxcoefftests/tiptoe_dense/0013/expec", sep='\s+', header=8)
#shift every column name by one and remove the first column
df_dense.columns = df_dense.columns[1:].tolist() + [""]
#remove last column
df_dense = df_dense.iloc[:, :-1]


time0 = extract_time("/home/user/BachelorThesis/trecxcoefftests/tiptoe_dense/0013/Laser")
field0 = extractField_0("/home/user/BachelorThesis/trecxcoefftests/tiptoe_dense/0013/Laser")

integrand = np.exp(1j* 1/2 * np.array(time0)) * np.array(field0) * np.exp(-1j* 1/2 * np.array(time0)) #because of interaction representation
cumulative_integral = cumulative_trapezoid(-1j*integrand, x=np.array(time0), initial=0)
interp_func = interp1d(time0, cumulative_integral, kind='cubic', fill_value="extrapolate")
cumulative_integral_reduced = interp_func(df_dense["Time"])



fig = go.Figure()
fig.add_trace(go.Scatter(x=np.array(df_dense["Time"]), y=np.array(df_dense["<Occ{H0:0}>"]), mode='lines', name='c^2_Occ_new'))
fig.add_trace(go.Scatter(x=np.array(df_dense["Time"]), y=1-(np.real(cumulative_integral_reduced)**2+np.imag(cumulative_integral_reduced)**2), mode='lines', name='c^2_analytical'))
#fig.add_trace(go.Scatter(x=np.array(df_dense["Time"]), y=np.array(df_dense["Re{<H0:2|psi>}"]**2+df_dense["Imag{<H0:2|psi>}"]**2), mode='lines', name='c^2_new'))
#fig.add_trace(go.Scatter(x=np.array(df_dense["Time"]), y=np.array(df_dense["Re{<H0:0|psi>}"]), mode='lines', name='c^2_new'))

# Laser Pulse

In [2]:
time0 = extract_time("/home/user/BachelorThesis/trecxcoefftests/tiptoe_short/0367/Laser")
field0 = extractField_0("/home/user/BachelorThesis/trecxcoefftests/tiptoe_short/0367/Laser")

def cos8Envelope(t, fwhm):
    return (np.cos(-t**2/ fwhm))**8

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.array(time0), y=np.array(field0), mode='lines', name='field0'))
#fig.add_trace(go.Scatter(x=np.array(time0), y=cos8Envelope(np.array(time0), 1e-15), mode='lines', name='cos8Envelope'))